In [2]:
%matplotlib inline
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import gru_ode_bayes 
import gru_ode_bayes.data_utils as data_utils
import time
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from gru_ode_bayes import Logger
import matplotlib.pyplot as plt

In [3]:
file_path="E:\\Data\\Private\\MIMIC_III\\mimic_csv\\Clean_data\\"
csv_file_path =file_path+"GRU_ODE_Dataset.csv"
covs_file_path=file_path+"GRU_ODE_covariates.csv"
tags_file_path=file_path+"GRU_ODE_death_tags.csv"

In [4]:
df=pd.read_csv(csv_file_path)
df.head()

,ID,Time,Value_0,Value_1,Value_2,Value_3,Value_4,Value_5,Value_6,Value_7,...,Mask_86,Mask_87,Mask_88,Mask_89,Mask_90,Mask_91,Mask_92,Mask_93,Mask_94,Mask_95
0,0.0,0.0,0.67391,-0.590107,0.859817,-1.231018,-0.450322,0.923803,-0.003803,-1.224005,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,9.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.817940,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,23.0,0.00000,0.000000,0.000000,0.000000,0.000000,1.209635,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,84.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,161.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
tags=pd.read_csv(tags_file_path)
tags

,ID,label
0,7513,0
1,10119,0
2,259,0
3,20877,0
4,3092,0
...,...,...
21252,2344,0
21253,4211,0
21254,20975,0
21255,21084,0


In [6]:
tags.columns

Index(['ID', 'label'], dtype='object')

In [7]:
covs=pd.read_csv(covs_file_path)

In [ ]:
tags.sum()

In [ ]:
covs.sum()

In [8]:
import pickle

def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data

In [9]:
# ロード
d=pickle_load( file_path+'sample_unique_id_dict.pickle')

In [10]:
idx=np.array(list(d.keys()))

In [65]:
# 全部使うと重いのでsample数少な目で実験

In [72]:
# df.iloc[0:5000,:]

,ID,Time,Value_0,Value_1,Value_2,Value_3,Value_4,Value_5,Value_6,Value_7,...,Mask_86,Mask_87,Mask_88,Mask_89,Mask_90,Mask_91,Mask_92,Mask_93,Mask_94,Mask_95
0,0.0,0.0,0.67391,-0.590107,0.859817,-1.231018,-0.450322,0.923803,-0.003803,-1.224005,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,9.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.817940,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,23.0,0.00000,0.000000,0.000000,0.000000,0.000000,1.209635,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,84.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,161.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,85.0,825.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,85.0,840.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,85.0,841.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,85.0,846.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
# idx=np.array(list(df.iloc[0:5000,:]["ID"].unique()))

In [69]:
# source=df.iloc[0:5000,:]

In [50]:
data = data_utils.ODE_Dataset(csv_file=csv_file_path,label_file=tags_file_path, cov_file=covs_file_path, idx=idx)
# data_val = data_utils.ODE_Dataset(csv_file=csv_file_path,label_file=None, cov_file=None, idx=idx,validation=True)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# val_option
# val_options  dictionnary with validation dataset options.
# T_val : Time after which observations are considered as test samples
# max_val_samples : maximum number of test observations per trajectory.
params_dict={}
params_dict["T_val"] = 150
params_dict["max_val_samples"] = 3
val_options = {"T_val": params_dict["T_val"], "max_val_samples": params_dict["max_val_samples"]}

In [51]:
# data = data_utils.ODE_Dataset( panda_df=source ,label_file=tags, cov_file=covs, idx=idx)
# # data_val = data_utils.ODE_Dataset(panda_df=source,label_file=None, cov_file=None, idx=idx,validation=True ,
# #                                  val_options=val_options)

In [52]:
dl   = DataLoader(dataset=data, collate_fn=data_utils.custom_collate_fn, shuffle=True, batch_size=1,num_workers=4)
# dl_val   = DataLoader(dataset=data_val, collate_fn=data_utils.custom_collate_fn, shuffle=True,batch_size=100,num_workers=4)

In [60]:
tmp = dl.__iter__()
b = tmp.next() 



In [54]:
len(tmp)

213

In [78]:
# validationまわし
# tmp = dl_val.__iter__()
# b = tmp.next() 


In [20]:
60*48

2880

In [61]:
b["times"]

array([ 0.,  8., 10., 29., 79.], dtype=float32)

In [62]:
b["time_ptr"]

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [63]:
len(b["obs_idx"])

5

In [64]:
for i in b["obs_idx"]:
    print(i)

tensor(0, dtype=torch.int32)
tensor(0, dtype=torch.int32)
tensor(0, dtype=torch.int32)
tensor(0, dtype=torch.int32)
tensor(0, dtype=torch.int32)


In [65]:
b["times"][-1]

79.0

In [67]:
b["X"]

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.5413,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, 

In [25]:
for i in b["obs_idx"]:
    print(i)

tensor(0, dtype=torch.int32)
tensor(53, dtype=torch.int32)
tensor(52, dtype=torch.int32)
tensor(23, dtype=torch.int32)
tensor(35, dtype=torch.int32)
tensor(51, dtype=torch.int32)
tensor(50, dtype=torch.int32)
tensor(34, dtype=torch.int32)
tensor(49, dtype=torch.int32)
tensor(3, dtype=torch.int32)
tensor(4, dtype=torch.int32)
tensor(56, dtype=torch.int32)
tensor(21, dtype=torch.int32)
tensor(5, dtype=torch.int32)
tensor(42, dtype=torch.int32)
tensor(22, dtype=torch.int32)
tensor(6, dtype=torch.int32)
tensor(2, dtype=torch.int32)
tensor(41, dtype=torch.int32)
tensor(27, dtype=torch.int32)
tensor(29, dtype=torch.int32)
tensor(55, dtype=torch.int32)
tensor(26, dtype=torch.int32)
tensor(30, dtype=torch.int32)
tensor(25, dtype=torch.int32)
tensor(33, dtype=torch.int32)
tensor(24, dtype=torch.int32)
tensor(36, dtype=torch.int32)
tensor(37, dtype=torch.int32)
tensor(32, dtype=torch.int32)
tensor(38, dtype=torch.int32)
tensor(39, dtype=torch.int32)
tensor(40, dtype=torch.int32)
tensor(31, dtype

In [79]:
b.keys()

dict_keys(['pat_idx', 'times', 'time_ptr', 'X', 'M', 'obs_idx', 'y', 'cov', 'X_val', 'M_val', 'times_val', 'index_val', 'X_last', 'obs_idx_last'])

In [32]:
b["obs_idx"]

tensor([ 0, 49, 50,  ...,  0, 35,  7], dtype=torch.int32)

In [28]:
len(b["pat_idx"])

100

In [35]:
b["X"].shape

torch.Size([7234, 96])

In [33]:
b["M"].shape

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [38]:
b["obs_idx"]

tensor([ 0, 61, 60,  ..., 96, 16, 36], dtype=torch.int32)

In [40]:
b["y"].shape

torch.Size([100, 1])

In [42]:
b["cov"].shape

torch.Size([100, 1])

In [47]:
b["X_val"]

res["pat_idx"] :  patients_index? sampleid

res["time_ptr"]:　

res["X"]: Value

res["M"]: Mask

res["obs_idx"]: </br>
    バッチ内でのidxといったところか</br>
    batch作成時にsampleをシャッフルするので、シャッフル前のバッチのidxみたいな感じ？</br>
    pat_idxに対して、batch内でidxを0からふりなおしているようなことをしている</br>


res["y"]: 1系列に1値、系列全体からの予測対象、おそらくdeath_tagがここにくるのだろう

res["cov"]:     

res["X_val"]: validation時のX?

res["M_val"]:　validation時のM?

res["times_val"]:　validation時のtime?

res["index_val"]: validation時のidx?

- val系 : data_utils.ODE_Datasetのオブジェクトを生成する際に引数validationをTrueに,val_optionsをdictで正しく指定すると数値等が入ってくるようだ。デフォルトはFalseでこのときはNoneになる

```python
# example
params_dict["T_val"] = 150
params_dict["max_val_samples"] = 3
val_options = {"T_val": params_dict["T_val"], "max_val_samples": params_dict["max_val_samples"]}
```

res["X_last"]:

res["obs_idx_last"]: